In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import torch
import seaborn as sns
import os
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

/home/arjun/AI_ENV/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


No need to use torch here. I just copy-pasted the nn template

# Dataset

In [2]:
loc = '/home/arjun/Desktop/Datasets/titanic'

In [3]:
df = pd.read_csv(loc + '/train.csv')

In [4]:
df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [5]:
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S


In [6]:
df['Sex'] = df['Sex'].replace({'male': 0, 'female': 1})
df['Embarked'] = df['Embarked'].replace({'S':1, 'C':2, 'Q':3})

In [7]:
df.fillna(df.mean(), inplace=True)
df.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,0,22.0,1,0,7.2500,1.0
1,2,1,1,1,38.0,1,0,71.2833,2.0
2,3,1,3,1,26.0,0,0,7.9250,1.0
3,4,1,1,1,35.0,1,0,53.1000,1.0
4,5,0,3,0,35.0,0,0,8.0500,1.0


In [8]:
y = df['Survived']
X = df.drop('Survived', axis=1)

In [9]:
X = torch.tensor(np.array(X))
y = torch.tensor(np.array(y))

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.5)
X_train.shape, X_test.shape ,y_train.shape, y_test.shape

(torch.Size([445, 8]),
 torch.Size([446, 8]),
 torch.Size([445]),
 torch.Size([446]))

# Model Creation

In [87]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, criterion='entropy', max_depth=4)
model.fit(X_train, y_train)
model.score(X_train, y_train), model.score(X_test, y_test)


(0.8651685393258427, 0.820627802690583)

# Training Model

In [68]:
model.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', max_depth=2, n_estimators=300)

# Checking Accuracy

In [65]:
model.score(X_train, y_train)

0.8449438202247191

In [66]:
model.score(X_test, y_test)

0.8139013452914798

# Running Test Dataset

In [88]:
df = pd.read_csv(loc + '/test.csv')
df = df.drop(['Name', 'Ticket', 'Cabin'], axis=1)
df['Sex'] = df['Sex'].replace({'male': 0, 'female': 1})
df['Embarked'] = df['Embarked'].replace({'S':1, 'C':2, 'Q':3})
df.fillna(df.mean(), inplace=True)
ds = torch.tensor(np.array(df))

In [89]:
l = model.predict(ds)

In [90]:

id_ = [i for i in range(892,1310 )]
ans = pd.DataFrame({'PassengerId':id_,'Survived':l})
ans.to_csv('./submission.csv', index=False)
print("Saved new O/P")
ans.head(20)

Saved new O/P


,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
5,897,0
6,898,0
7,899,0
8,900,1
9,901,0
